# CNN Transfer&Train&Predict

# Init Config Setup

In [2]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00011"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

# get Active workflow version
wf_ver_id = 0
if wf_ver_id == 0:
    resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/')
    data = json.loads(resp.json())

    # get Max workflow version
    for config in data:
        if config["fields"]["nn_wf_ver_id"] > wf_ver_id:
            wf_ver_id = config["fields"]["nn_wf_ver_id"]

wf_ver_id = str(wf_ver_id)

print("Network ID="+nn_id)
print("Workflow Version="+wf_ver_id)
print("")

def spaceprint(val, cnt):
    leng = len(str(val))
    cnt = cnt - leng
    restr = ""
    for i in range(cnt):
        restr += " "
    restr = restr+str(val)
    return restr

node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "param":{"epoch":1
                                  ,"traincnt": 2,
                                     "batch_size":150,
                                     "predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                     "layeroutputs":32,
                                     "type":"category"
                                     }
                         ,"layer1": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                          ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
        ,"labels":[]
                        })
netconf = json.loads(resp.json())
print("Network Config : {0}".format(netconf))
print("")

node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         

                     })
dataconf = json.loads(resp.json())
print("Train Data Config : {0}".format(dataconf))
print("")

node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("Eval Config : {0}".format(evalconf))
print("")

node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}

                     })
edataconf = json.loads(resp.json())
print("Eval Data Config : {0}".format(edataconf))
print("")

Network ID=nn00011
Workflow Version=2

Network Config : {'config': {'num_classes': 10, 'type': 'category', 'learnrate': 0.001, 'layeroutputs': 32}, 'param': {'traincnt': 2, 'epoch': 1, 'predictcnt': 10, 'batch_size': 150}, 'layer1': {'maxpoolstride': [2, 2], 'layercnt': 2, 'cnnfilter': [3, 3], 'padding': 'SAME', 'droprate': '0.8', 'maxpoolmatrix': [2, 2], 'type': 'cnn', 'cnnstride': [1, 1], 'active': 'relu'}, 'layer2': {'maxpoolstride': [2, 2], 'layercnt': 1, 'cnnfilter': [3, 3], 'padding': 'SAME', 'droprate': '0.8', 'maxpoolmatrix': [2, 2], 'type': 'cnn', 'cnnstride': [1, 1], 'active': 'relu'}, 'labels': [], 'out': {'node_out': 625, 'active': 'softmax', 'padding': 'SAME'}}

Train Data Config : {'preprocess': {'channel': 3, 'x_size': 32, 'y_size': 32}, 'source_path': '/hoya_src_root/nn00011/2/datasrc', 'store_path': '/hoya_str_root/nn00011/2/datasrc'}

Eval Config : {}

Eval Data Config : {'preprocess': {'channel': 3, 'x_size': 32, 'y_size': 32}, 'source_path': '/hoya_src_root/nn00011/

# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

아래 API의 경우 Test Data인지 Train Data인지 선택할수 있는 Node 정보를 모두 가져와 준다.

In [3]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

get file node list
{'nn_wf_node_name': 'evaldata', 'nn_wf_node_desc': 'Evaluation Data Node', 'nn_wf_node_id': 'nn00011_2_evaldata'}
{'nn_wf_node_name': 'datasrc', 'nn_wf_node_desc': 'Train Data Node', 'nn_wf_node_id': 'nn00011_2_datasrc'}



아래 API는 한개의 파일을 Train Folder & Test Folder 에 넣어준다.

In [4]:
# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

create file node list
{'File0': {'File': 'File Create.(/hoya_src_root/nn00011/2/evaldata/sample_cnn_img.zip)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00011/2/evaldata/sample_cnn_img.zip)'}}
{'File0': {'File': 'File Create.(/hoya_src_root/nn00011/2/datasrc/sample_cnn_img.zip)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00011/2/datasrc/sample_cnn_img.zip)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br> 

In [5]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result":
        try:
            for tr in train["TrainResult"]:
                print(tr)
        except:
            maxcnt = 0
            line = ""
            for label in train["labels"]:
                if maxcnt<len(label):
                    maxcnt = len(label)

            for i in range(len(train["labels"])):
                for j in range(maxcnt+4):
                    line += "="

            label_sub = []
            for label in train["labels"]:
                label = spaceprint(label,maxcnt)
                label_sub.append(label)

            space = ""
            for s in range(maxcnt):
                space +=" "

            print(space, label_sub)
            print(space, line)
            for i in range(len(train["labels"])):
                truecnt = 0
                totcnt = 0
                predict_sub = []
                for j in range(len(train["predicts"][i])):
                    pred = spaceprint(train["predicts"][i][j],maxcnt)

                    predict_sub.append(pred)
                    totcnt += int(pred)
                    # print(train["labels"].index(train["labels"][i]))
                    if train["labels"].index(train["labels"][i]) == j:
                        truecnt = int(pred)
                if totcnt == 0:
                    percent = 0
                else:
                    percent = round(truecnt/totcnt*100,2)
                print(spaceprint(train["labels"][i],maxcnt), predict_sub, str(percent)+"%")

['Trainning ..................................................']
['Global Step: 12, Training Batch Accuracy: 17.65%, Cost: 244.769']

         ['   motor', '     dog', '     car', '   glove', 'airplane', '     cat', '    bolt']
   motor ['       0', '       0', '       2', '       0', '       1', '       0', '       0'] 0.0%
     dog ['       0', '       0', '       1', '       0', '       0', '       0', '       0'] 0.0%
     car ['       0', '       0', '       3', '       0', '       0', '       0', '       0'] 100.0%
   glove ['       0', '       0', '       1', '       0', '       1', '       0', '       1'] 0.0%
airplane ['       0', '       0', '       2', '       0', '       1', '       0', '       0'] 33.33%
     cat ['       0', '       0', '       1', '       0', '       0', '       0', '       0'] 0.0%
    bolt ['       0', '       0', '       1', '       0', '       1', '       0', '       1'] 33.33%


# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [6]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')

FileName = 2car.jpg
['car', 'cat', 'bolt', 'dog', 'airplane', 'glove', 'motor']
[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

FileName = 1air.jpg
['bolt', 'car', 'airplane', 'dog', 'glove', 'cat', 'motor']
[99.999988, 1.4999999999999999e-05, 0.0, 0.0, 0.0, 0.0, 0.0]

FileName = 2air.jpg
['car', 'airplane', 'bolt', 'glove', 'dog', 'cat', 'motor']
[92.748576, 4.583563, 2.6678629999999997, 0.0, 0.0, 0.0, 0.0]

FileName = 1glove.jpg
['airplane', 'bolt', 'glove', 'dog', 'car', 'cat', 'motor']
[100.0, 1e-06, 0.0, 0.0, 0.0, 0.0, 0.0]

FileName = 2motor.jpg
['airplane', 'bolt', 'car', 'cat', 'glove', 'dog', 'motor']
[73.07287500000001, 26.695204, 0.231923, 0.0, 0.0, 0.0, 0.0]

FileName = 2glove.jpg
['glove', 'car', 'airplane', 'cat', 'dog', 'bolt', 'motor']
[99.855965, 0.117309, 0.026729, 0.0, 0.0, 0.0, 0.0]

FileName = 1motor.jpg
['airplane', 'bolt', 'cat', 'glove', 'car', 'dog', 'motor']
[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

FileName = 1car.jpg
['car', 'bolt', 'cat', 'airplane', 'glove', 'dog', 'm